In [1]:
import numpy
import pandas
import geopandas
import pysal
import seaborn
import contextily
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import csv
import shapely
from shapely.geometry import Point, GeometryCollection
from libpysal.weights import Queen, Rook, KNN
import libpysal

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_5076/219099670.py:3: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS di

Making Grid Neighbourhoods

In [3]:
schools = pandas.read_csv("/home/azureuser/cloudfiles/code/Users/ariley/Data/Schools/OSM_All_Merge/NE_schools_wGHSL.csv")

schools = schools.sort_values('lat')
schools.head()

/tmp/ipykernel_19642/188732367.py:1: DtypeWarning: Columns (40,46) have mixed types. Specify dtype option on import or set low_memory=False.
  schools = pandas.read_csv("/home/azureuser/cloudfiles/code/Users/ariley/Data/Schools/OSM_All_Merge/NE_schools_wGHSL.csv")


,Unnamed: 0,DEGURBA_L1_adm1,GID_2,GID_0_adm1,GID_1,GID_0_adm2,COUNTRY_adm2,NAME_1_adm2,NL_NAME_1_adm2,VARNAME_2,...,Distance,dist,land,pop,built_s,built_v,smod,NAME_2,lon,lat
20746,34131,2,BRA.13.233_2,BRA,BRA.13_1,BRA,Brazil,Minas Gerais,NaN,NaN,...,NaN,NaN,10000,71.236717,2932,7330,21,Curvelo,-4.314070e+06,-2.253773e+06
26817,55520,2,BRA.13.752_2,BRA,BRA.13_1,BRA,Brazil,Minas Gerais,NaN,NaN,...,NaN,NaN,10000,14.743477,1212,3030,12,São Pedro do Suaçuí,-4.129290e+06,-2.253635e+06
20747,34141,2,BRA.13.233_2,BRA,BRA.13_1,BRA,Brazil,Minas Gerais,NaN,NaN,...,NaN,NaN,10000,91.820534,3779,9448,21,Curvelo,-4.314188e+06,-2.253629e+06
20745,34111,2,BRA.13.233_2,BRA,BRA.13_1,BRA,Brazil,Minas Gerais,NaN,NaN,...,NaN,NaN,10000,51.488697,2119,5298,21,Curvelo,-4.314188e+06,-2.253594e+06
21452,36069,2,BRA.13.300_2,BRA,BRA.13_1,BRA,Brazil,Minas Gerais,NaN,NaN,...,NaN,NaN,10000,8.316339,816,2040,11,Frei Inocêncio,-4.066675e+06,-2.253521e+06


In [4]:
gdf = geopandas.GeoDataFrame(schools, 
            geometry=geopandas.points_from_xy(schools.lon, schools.lat),
            crs="+proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs")
gdf.head()
gdf = gdf.drop(columns = ['lon', 'lat'])

xmin, ymin, xmax, ymax= gdf.total_bounds

print(xmin, ymin, xmax, ymax)

-4873230.6002699 -2253773.45274861 -3243566.23593232 -129745.902688861


In [3]:
rW = Rook.from_shapefile("/home/azureuser/cloudfiles/code/Users/ariley/Data/Schools/School_Counts/ne_brazil_school_density_1km.shp") #, to_file = "/home/azureuser/cloudfiles/code/Users/ariley/Data/Schools/School_Counts/ne_grid_weights.shp")


#schools.hist(column = 'NUMPOINTS', bins = 10)

#ax = schools.plot(column='NUMPOINTS', figsize=(12, 8), cmap='viridis', vmax=5000, edgecolor="grey")

TypeError: 'Rook' object is not callable